My traditional GP clustering - emulates TSNE using Kullback Leibler
Might be useful as a connection from training data to test

In [ ]:
import gc
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
def get_inputs(data, metadata):
    metadata = metadata.copy()
    data = data.copy()
    metadata.drop(['ra','decl','gal_l','gal_b','distmod'],inplace=True,axis=1)
    
    data['flux_ratio_sq'] = np.power(data['flux'] / data['flux_err'], 2.0)
    data['flux_by_flux_ratio_sq'] = data['flux'] * data['flux_ratio_sq']
    aggdata = data.copy().groupby(['object_id','passband']).agg({'mjd': ['min', 'max', 'size'],
                                                         'flux': ['min', 'max', 'mean', 'median', 'std','skew'],
                                                         'flux_err': ['min', 'max', 'mean', 'median', 'std','skew'],
                                                         'flux_by_flux_ratio_sq': ['sum'],    
                                                         'flux_ratio_sq': ['sum'],                      
                                                         'detected': ['mean','std']}).reset_index(drop=False)
    
    cols = ['_'.join(str(s).strip() for s in col if s) if len(col)==2 else col for col in aggdata.columns ]
    aggdata.columns = cols
    aggdata = aggdata.merge(metadata,on='object_id',how='left')
    aggdata.insert(1,'delta_passband', aggdata.mjd_max-aggdata.mjd_min)
    aggdata.drop(['mjd_min','mjd_max'],inplace=True,axis=1)
    aggdata['flux_diff'] = aggdata['flux_max'] - aggdata['flux_min']
    aggdata['flux_dif2'] = (aggdata['flux_max'] - aggdata['flux_min']) / aggdata['flux_mean']
    aggdata['flux_w_mean'] = aggdata['flux_by_flux_ratio_sq_sum'] / aggdata['flux_ratio_sq_sum']
    aggdata['flux_dif3'] = (aggdata['flux_max'] - aggdata['flux_min']) / aggdata['flux_w_mean']
    detaggdata = data[data.detected==1].copy().groupby(['object_id','passband']).agg({'mjd': ['min', 'max', 'size'],
                                                     'flux': ['min', 'max', 'mean', 'median', 'std','skew'],
                                                     'flux_err': ['min', 'max', 'mean', 'median', 'std','skew'],
                                                     'flux_by_flux_ratio_sq': ['sum'],    
                                                     'flux_ratio_sq': ['sum']}).reset_index(drop=False)

    cols = ['_'.join(str(s).strip() for s in col if s) if len(col)==2 else col for col in detaggdata.columns ]
    detaggdata.columns = cols
    detaggdata = detaggdata.merge(metadata,on='object_id',how='left')
    detaggdata.insert(1,'delta_passband', detaggdata.mjd_max-detaggdata.mjd_min)
    detaggdata.drop(['mjd_min','mjd_max'],inplace=True,axis=1)
    detaggdata['flux_diff'] = detaggdata['flux_max'] - detaggdata['flux_min']
    detaggdata['flux_dif2'] = (detaggdata['flux_max'] - detaggdata['flux_min']) / detaggdata['flux_mean']
    detaggdata['flux_w_mean'] = detaggdata['flux_by_flux_ratio_sq_sum'] / detaggdata['flux_ratio_sq_sum']
    detaggdata['flux_dif3'] = (detaggdata['flux_max'] - detaggdata['flux_min']) / detaggdata['flux_w_mean']
    detaggdata.columns = ['det_'+col if (col not in ['object_id','passband']) else col for col in detaggdata.columns  ]
    if('det_target' in detaggdata.columns):
        detaggdata.drop('det_target',inplace=True,axis=1)
    return aggdata.merge(detaggdata,on=['object_id','passband'],how='left')


In [ ]:
meta_train = pd.read_csv('../input/training_set_metadata.csv')
train = pd.read_csv('../input/training_set.csv')
traindata = get_inputs(train,meta_train)

In [ ]:
features = ['mjd_size', 'flux_median', 'flux_err_max', 'flux_diff', 'det_flux_w_mean',
             'det_flux_dif3', 'det_flux_median', 'delta_passband',  'flux_skew',
             'det_flux_err_mean', 'flux_ratio_sq_sum', 'det_flux_err_max', 'flux_err_mean',
             'flux_by_flux_ratio_sq_sum', 'det_hostgal_photoz', 'ddf', 'det_mwebv',
             'flux_err_skew', 'mwebv', 'flux_dif2', 'det_delta_passband', 'det_hostgal_photoz_err',
             'flux_std', 'flux_max', 'flux_min', 'hostgal_photoz', 'flux_mean',
             'det_flux_err_min', 'det_hostgal_specz', 'det_flux_ratio_sq_sum', 'det_ddf',
             'det_flux_min', 'det_mjd_size', 'det_flux_by_flux_ratio_sq_sum', 'det_flux_mean',
             'det_flux_err_median', 'flux_err_min', 'flux_err_median', 'detected_mean',
             'flux_w_mean', 'det_flux_skew', 'det_flux_std', 'det_flux_max', 'hostgal_photoz_err',
             'det_flux_diff', 'det_flux_err_std', 'flux_err_std', 'detected_std',
             'flux_dif3', 'det_flux_dif2', 'passband', 'det_flux_err_skew']
allfeatures = ['object_id']+features

In [ ]:
ssmean = np.array([ 3.49771025,  0.42019642,  2.38524252,  3.84729773,  3.03272983,
                    0.26669406,  2.85494442,  6.75841373,  0.55194974,  1.60952193,
                    5.118674  ,  1.75876685,  1.82327962,  5.59201149,  0.57884157,
                    0.47915551,  0.05876913,  0.62022843,  0.05686373,  1.85592118,
                    2.59708769,  0.14567936,  2.48444712,  3.38288084, -2.57052464,
                    0.62374943,  0.95878505,  1.44482923,  0.37546532,  5.92922363,
                    0.5146859 ,  2.10058762,  1.4456725 ,  7.84078499,  2.94307062,
                    1.59223645,  1.36668414,  1.76598399,  0.08498087,  1.99068911,
                    0.03999367,  2.79567961,  3.64814012,  0.14237247,  2.44718014,
                    0.75461085,  1.08367222,  0.14581964,  0.87834825,  0.31520748,
                    2.50000114,  0.31771039])
ssscale = np.array([0.58642916, 1.58822583, 1.03230677, 1.35906529, 1.884692  ,
                   0.50631095, 1.89879334, 0.10698923, 0.68585523, 0.67946487,
                   1.98117288, 0.72998711, 0.92238339, 6.21324731, 0.35518699,
                   0.49956532, 0.09669229, 0.51376129, 0.12357035, 2.67054438,
                   1.57177736, 0.19088875, 1.35332701, 1.467286  , 1.37966648,
                   0.4712452 , 1.73821205, 0.61014514, 0.11695824, 1.60077377,
                   0.36611121, 2.23753884, 0.54579949, 4.71697785, 1.7914595 ,
                   0.67699036, 0.79762004, 0.92793836, 0.18462479, 2.44210838,
                   0.33493065, 1.15078837, 1.4861533 , 0.25086545, 1.71601522,
                   0.52575798, 0.78775387, 0.15563803, 1.70852628, 0.60803242,
                   1.70782691, 0.3152083 ])

In [ ]:
alldata = traindata.loc[:,allfeatures].copy()
for c in features:
    print(c)
    if(alldata[c].min()<0):
        alldata.loc[~alldata[c].isnull(),c] = np.sign(alldata.loc[~alldata[c].isnull(),c])*np.log1p(np.abs(alldata.loc[~alldata[c].isnull(),c]))
    elif((alldata[c].max()-alldata[c].min())>10):
        alldata.loc[~alldata[c].isnull(),c] = np.log1p(alldata.loc[~alldata[c].isnull(),c])
alldata.fillna(alldata.mean(),inplace=True)

ss = StandardScaler()
ss.mean_ = ssmean
ss.scale_ = ssscale
alldata.loc[:,features] = ss.transform(alldata.loc[:,features])
print(ss.mean_)
print(ss.scale_)

In [ ]:
alldata[features].head()

In [ ]:
def GPx(data):
    return (1.0*np.tanh(((((((np.maximum(((data["flux_w_mean"])), ((data["flux_dif3"])))) - (data["flux_err_max"]))) * 2.0)) * 2.0)) +
            1.0*np.tanh(np.where(data["flux_ratio_sq_sum"]>0, np.minimum(((np.minimum(((data["flux_w_mean"])), ((data["flux_err_mean"]))))), ((data["flux_err_mean"]))), ((data["flux_by_flux_ratio_sq_sum"]) - (data["flux_err_std"])) )) +
            1.0*np.tanh(((np.where(data["flux_ratio_sq_sum"]>0, np.minimum(((((data["flux_std"]) * 2.0))), ((data["flux_by_flux_ratio_sq_sum"]))), data["flux_by_flux_ratio_sq_sum"] )) * 2.0)) +
            1.0*np.tanh(((((((np.minimum(((data["flux_std"])), ((data["flux_err_max"])))) * (data["flux_by_flux_ratio_sq_sum"]))) - (data["flux_err_skew"]))) + (((data["flux_max"]) * (data["flux_by_flux_ratio_sq_sum"]))))) +
            1.0*np.tanh(((np.tanh((data["flux_err_max"]))) * ((-1.0*((data["flux_err_max"])))))) +
            1.0*np.tanh(np.where(data["flux_ratio_sq_sum"] > -1, ((2.0) - (((data["flux_err_skew"]) * (((data["flux_err_skew"]) / 2.0))))), data["flux_max"] )) +
            1.0*np.tanh(np.where(data["flux_ratio_sq_sum"]>0, data["flux_err_median"], np.where(data["flux_ratio_sq_sum"]>0, data["flux_err_std"], (-1.0*((data["flux_err_std"]))) ) )) +
            1.0*np.tanh(((np.where(data["flux_err_max"]<0, data["flux_max"], np.where(data["flux_err_std"]>0, ((((data["flux_mean"]) * (data["flux_ratio_sq_sum"]))) / 2.0), data["flux_err_max"] ) )) * 2.0)) +
            1.0*np.tanh(np.where(data["flux_ratio_sq_sum"] > -1, np.where(data["flux_ratio_sq_sum"] > -1, (-1.0*((np.maximum(((data["flux_err_std"])), ((data["flux_ratio_sq_sum"])))))), data["flux_mean"] ), data["flux_err_std"] )) +
            1.0*np.tanh(np.where((((data["flux_w_mean"]) < (data["flux_dif3"]))*1.)>0, np.where((((data["flux_mean"]) < (data["flux_by_flux_ratio_sq_sum"]))*1.)>0, data["flux_dif3"], data["flux_ratio_sq_sum"] ), data["flux_std"] )) +
            0.996580*np.tanh(np.where(data["flux_w_mean"]<0, np.tanh(((9.0))), (((data["flux_dif3"]) < (((((data["flux_err_median"]) * (data["flux_dif3"]))) * (data["flux_by_flux_ratio_sq_sum"]))))*1.) )) +
            1.0*np.tanh(np.where(data["flux_err_min"]<0, data["flux_err_skew"], np.where(data["flux_err_min"]<0, np.tanh((data["flux_err_min"])), (-1.0*((np.maximum(((data["flux_max"])), ((data["flux_err_skew"])))))) ) )) +
            1.0*np.tanh(np.where(data["flux_ratio_sq_sum"] > -1, np.where(data["flux_ratio_sq_sum"] > -1, ((data["flux_by_flux_ratio_sq_sum"]) * (np.minimum(((data["flux_err_std"])), ((data["flux_err_mean"]))))), data["flux_ratio_sq_sum"] ), data["flux_ratio_sq_sum"] )) +
            1.0*np.tanh(np.where((-1.0*((data["flux_err_max"])))>0, data["flux_mean"], np.where((-1.0*((data["flux_err_median"]))) > -1, (-1.0*((((data["flux_mean"]) * 2.0)))), data["flux_err_max"] ) )) +
            1.0*np.tanh(np.where(((0.636620) + (data["flux_ratio_sq_sum"]))<0, np.where(0.636620<0, -1.0, data["flux_ratio_sq_sum"] ), (((data["flux_ratio_sq_sum"]) < (data["flux_mean"]))*1.) )) +
            1.0*np.tanh(np.where(data["flux_dif3"]<0, data["flux_dif2"], ((data["flux_err_std"]) * (((data["flux_min"]) + ((-1.0*((data["flux_dif2"]))))))) )) +
            1.0*np.tanh(((((((((((data["flux_err_median"]) > (((data["flux_err_min"]) / 2.0)))*1.)) > (data["flux_err_median"]))*1.)) + (((data["flux_err_median"]) - (data["flux_by_flux_ratio_sq_sum"]))))) * 2.0)) +
            1.0*np.tanh(np.where(data["flux_err_max"] > -1, (-1.0*((np.where(data["flux_err_min"]>0, ((data["flux_max"]) * 2.0), (-1.0*((data["flux_mean"]))) )))), (-1.0*((data["flux_max"]))) )) +
            1.0*np.tanh(np.where(data["flux_ratio_sq_sum"] > -1, (((((data["flux_dif3"]) * (data["flux_err_median"]))) + (data["flux_ratio_sq_sum"]))/2.0), ((data["flux_err_min"]) * 2.0) )) +
            1.0*np.tanh((((-1.0*((data["flux_err_skew"])))) * ((((data["flux_err_skew"]) > (np.maximum(((1.570796)), (((-1.0*((np.maximum(((data["flux_err_skew"])), ((data["flux_err_skew"])))))))))))*1.)))) +
            1.0*np.tanh(np.minimum(((np.where(data["flux_min"] > -1, data["flux_min"], (((3.0) + (data["flux_err_skew"]))/2.0) ))), ((np.maximum(((data["flux_dif3"])), ((data["flux_err_skew"]))))))) +
            1.0*np.tanh(np.where(data["flux_median"] > -1, ((data["flux_ratio_sq_sum"]) * ((-1.0*((data["flux_ratio_sq_sum"]))))), (((-1.0) > (np.minimum(((data["flux_median"])), ((data["flux_median"])))))*1.) )) +
            1.0*np.tanh((-1.0*((((data["flux_dif2"]) * (np.minimum(((data["flux_std"])), (((((np.maximum(((data["flux_dif2"])), ((data["flux_std"])))) < (data["flux_err_max"]))*1.)))))))))) +
            1.0*np.tanh(((data["flux_dif3"]) * (np.minimum(((data["flux_err_max"])), ((np.where(data["flux_mean"]>0, data["flux_by_flux_ratio_sq_sum"], (((((data["flux_dif3"]) + (data["flux_std"]))/2.0)) / 2.0) ))))))) +
            1.0*np.tanh(((((((np.minimum(((data["flux_err_median"])), ((data["flux_w_mean"])))) > (np.tanh((data["flux_err_min"]))))*1.)) > (0.318310))*1.)) +
            1.0*np.tanh(np.where(data["flux_mean"] > -1, 0.0, np.where(data["flux_dif3"] > -1, 1.570796, np.where(3.0<0, 0.0, data["flux_dif3"] ) ) )) +
            1.0*np.tanh((-1.0*((((data["flux_err_std"]) * (np.where(data["det_flux_err_mean"] > -1, data["flux_dif2"], data["flux_diff"] ))))))) +
            1.0*np.tanh(((((data["flux_mean"]) * (((data["flux_max"]) - ((((((data["flux_max"]) / 2.0)) > (0.0))*1.)))))) / 2.0)) +
            1.0*np.tanh(np.tanh((((((np.where(data["flux_diff"]>0, (-1.0*((data["flux_dif2"]))), np.where(data["flux_diff"]>0, data["flux_err_mean"], data["flux_dif2"] ) )) / 2.0)) / 2.0)))) +
            1.0*np.tanh(((((data["flux_dif3"]) * (((np.where(data["flux_median"]>0, ((data["flux_by_flux_ratio_sq_sum"]) / 2.0), 0.636620 )) * (data["flux_err_min"]))))) * 2.0)) +
            1.0*np.tanh((((np.where(np.where((0.0)>0, data["flux_err_max"], -1.0 ) > -1, data["flux_err_mean"], data["flux_err_max"] )) < (-1.0))*1.)) +
            0.946751*np.tanh(np.where((((data["flux_by_flux_ratio_sq_sum"]) + ((-1.0*(((((data["flux_by_flux_ratio_sq_sum"]) < (data["flux_w_mean"]))*1.))))))/2.0)<0, 0.0, (-1.0*((data["flux_by_flux_ratio_sq_sum"]))) )) +
            1.0*np.tanh(((data["flux_dif3"]) * (np.where(data["flux_err_min"] > -1, (((((data["flux_min"]) > ((((data["flux_err_mean"]) > (data["flux_err_median"]))*1.)))*1.)) / 2.0), data["flux_err_mean"] )))) +
            1.0*np.tanh(np.where(np.where(data["flux_err_min"]<0, ((data["flux_std"]) * 2.0), data["flux_err_min"] ) > -1, (((np.tanh((data["flux_std"]))) > (data["flux_err_min"]))*1.), data["flux_err_skew"] )) +
            1.0*np.tanh(((np.where(data["flux_ratio_sq_sum"]<0, ((((data["flux_max"]) / 2.0)) * (data["flux_max"])), (-1.0*((data["flux_max"]))) )) / 2.0)) +
            1.0*np.tanh(((data["flux_std"]) - (data["flux_max"]))) +
            1.0*np.tanh(np.where(data["flux_dif3"]>0, 0.0, (((data["flux_mean"]) + (np.where(data["flux_err_median"]<0, ((data["flux_max"]) * (data["flux_err_median"])), data["flux_mean"] )))/2.0) )) +
            1.0*np.tanh((((((data["flux_err_mean"]) * (data["flux_err_min"]))) > (((data["flux_by_flux_ratio_sq_sum"]) + (3.0))))*1.)) +
            0.845139*np.tanh(((((((data["flux_dif3"]) * (((((-1.0*((data["flux_err_min"])))) < (0.636620))*1.)))) * (data["flux_err_skew"]))) * 2.0)) +
            1.0*np.tanh(((data["flux_dif3"]) * ((((data["flux_mean"]) < (np.minimum(((np.where(data["flux_err_min"] > -1, (-1.0*((data["flux_ratio_sq_sum"]))), data["flux_err_max"] ))), ((data["flux_ratio_sq_sum"])))))*1.)))) +
            1.0*np.tanh(np.where(0.0 > -1, (-1.0*((((((1.90740513801574707)) < (data["flux_err_skew"]))*1.)))), (-1.0*(((((1.570796) < (data["flux_err_skew"]))*1.)))) )) +
            1.0*np.tanh((((np.maximum(((data["flux_by_flux_ratio_sq_sum"])), ((((data["flux_max"]) / 2.0))))) < (((data["flux_w_mean"]) + (np.minimum(((((data["flux_ratio_sq_sum"]) / 2.0))), ((0.0)))))))*1.)) +
            1.0*np.tanh(np.where(data["flux_err_mean"]>0, (-1.0*((data["flux_max"]))), (((np.tanh(((-1.0*((3.0)))))) < (((data["flux_err_mean"]) * 2.0)))*1.) )) +
            0.991207*np.tanh((-1.0*(((((((data["flux_by_flux_ratio_sq_sum"]) > (data["flux_err_min"]))*1.)) + (((data["flux_err_std"]) * (((data["flux_dif2"]) + (data["flux_min"])))))))))) +
            1.0*np.tanh(np.where(((data["flux_err_median"]) / 2.0)<0, (((((data["flux_dif3"]) < (data["flux_err_skew"]))*1.)) / 2.0), (-1.0*((((((2.39746975898742676)) < (data["flux_err_skew"]))*1.)))) )) +
            1.0*np.tanh((((((3.141593) < (((((0.636620) - (data["flux_max"]))) - (data["flux_max"]))))*1.)) / 2.0)) +
            1.0*np.tanh((-1.0*(((((np.where((((0.318310) + ((((0.0) + (data["flux_ratio_sq_sum"]))/2.0)))/2.0)<0, data["flux_ratio_sq_sum"], (8.0) )) < (data["flux_std"]))*1.))))) +
            0.954568*np.tanh(((((((np.where(data["flux_err_std"]>0, (2.41745042800903320), data["flux_dif2"] )) + (data["flux_min"]))) / 2.0)) / 2.0)) +
            1.0*np.tanh(((((-1.0*((data["flux_by_flux_ratio_sq_sum"])))) + (((data["flux_by_flux_ratio_sq_sum"]) * ((((((data["flux_err_min"]) * (1.0))) < (data["flux_err_median"]))*1.)))))/2.0)) +
            1.0*np.tanh(((((((data["flux_min"]) * ((-1.0*((data["flux_max"])))))) - (data["flux_dif2"]))) * (data["flux_dif3"]))) +
            1.0*np.tanh(((np.maximum((((-1.0*((data["flux_err_mean"]))))), ((data["flux_err_median"])))) * ((-1.0*((((data["flux_by_flux_ratio_sq_sum"]) / 2.0))))))) +
            0.999511*np.tanh(np.where(np.tanh(((((data["flux_err_min"]) + ((-1.0*((data["flux_err_mean"])))))/2.0)))>0, 0.0, np.where(data["flux_ratio_sq_sum"]>0, data["flux_ratio_sq_sum"], data["flux_ratio_sq_sum"] ) )) +
            1.0*np.tanh(np.minimum((((-1.0*((((data["flux_ratio_sq_sum"]) * ((((data["flux_err_median"]) > ((-1.0*((data["flux_err_skew"])))))*1.)))))))), ((0.0)))) +
            1.0*np.tanh(((((((((data["flux_err_std"]) + (1.0))/2.0)) < ((((data["flux_mean"]) > (data["flux_ratio_sq_sum"]))*1.)))*1.)) / 2.0)) +
            1.0*np.tanh(np.minimum(((np.where(data["flux_err_skew"]<0, ((data["flux_diff"]) * (0.0)), data["flux_max"] ))), (((((data["flux_err_min"]) < (np.tanh((data["flux_err_skew"]))))*1.))))) +
            1.0*np.tanh(np.where(data["flux_err_min"] > -1, 0.0, ((np.where(data["flux_dif3"] > -1, (0.0), ((0.0) - (data["flux_dif3"])) )) - (data["flux_dif3"])) )) +
            1.0*np.tanh(((((np.where(data["flux_err_std"] > -1, ((data["flux_err_skew"]) * ((-1.0*((data["flux_mean"]))))), data["flux_dif3"] )) / 2.0)) / 2.0)) +
            1.0*np.tanh(((((((data["flux_by_flux_ratio_sq_sum"]) + (data["flux_err_median"]))/2.0)) < (-1.0))*1.)) +
            1.0*np.tanh((((((data["flux_ratio_sq_sum"]) < (data["flux_max"]))*1.)) * (((((data["flux_ratio_sq_sum"]) / 2.0)) * (data["flux_dif2"]))))) +
            1.0*np.tanh(((np.where(data["flux_by_flux_ratio_sq_sum"]<0, np.where(data["flux_err_median"]<0, data["flux_err_skew"], (((data["flux_err_median"]) + (data["flux_dif3"]))/2.0) ), np.tanh((data["flux_err_skew"])) )) / 2.0)) +
            1.0*np.tanh(np.where(data["flux_err_max"] > -1, 0.0, ((data["flux_dif3"]) * 2.0) )) +
            0.999511*np.tanh((((2.0) < (((((data["flux_err_skew"]) / 2.0)) - (data["flux_std"]))))*1.)) +
            1.0*np.tanh((((np.maximum(((((data["flux_err_mean"]) * (0.636620)))), ((1.570796)))) < (((((data["flux_err_mean"]) * (data["flux_w_mean"]))) - (data["flux_err_max"]))))*1.)) +
            1.0*np.tanh(((np.where(data["flux_ratio_sq_sum"]<0, data["flux_by_flux_ratio_sq_sum"], ((3.141593) * (((data["flux_err_std"]) - (data["flux_err_max"])))) )) / 2.0)) +
            1.0*np.tanh(np.where(data["flux_ratio_sq_sum"] > -1, np.where(data["flux_err_min"] > -1, 0.0, (-1.0*((data["flux_err_skew"]))) ), (-1.0*((data["flux_err_skew"]))) )) +
            1.0*np.tanh((((-1.0*((data["flux_dif2"])))) * ((((((data["flux_mean"]) > (data["flux_err_min"]))*1.)) * (data["flux_std"]))))) +
            1.0*np.tanh((((-1.0*((((0.318310) * (((data["flux_by_flux_ratio_sq_sum"]) + (((((12.58453464508056641)) > (((data["flux_err_min"]) + (0.318310))))*1.))))))))) / 2.0)) +
            0.877382*np.tanh(((data["flux_mean"]) * ((((np.where(((data["flux_mean"]) / 2.0) > -1, data["flux_by_flux_ratio_sq_sum"], data["flux_max"] )) > (data["flux_err_mean"]))*1.)))) +
            1.0*np.tanh((-1.0*(((((np.where(data["flux_mean"]<0, data["flux_err_max"], data["flux_err_skew"] )) > (2.0))*1.))))) +
            1.0*np.tanh((((((data["flux_ratio_sq_sum"]) < (data["flux_dif3"]))*1.)) * ((((1.0) < (np.where(data["flux_by_flux_ratio_sq_sum"]>0, data["flux_err_skew"], ((data["flux_by_flux_ratio_sq_sum"]) / 2.0) )))*1.)))) +
            1.0*np.tanh(((np.where(data["flux_err_mean"]<0, ((data["flux_err_std"]) + ((((data["flux_err_mean"]) < (data["flux_err_min"]))*1.))), (((1.570796) < (data["flux_err_min"]))*1.) )) / 2.0)) +
            1.0*np.tanh((((np.maximum(((data["flux_err_median"])), ((data["flux_by_flux_ratio_sq_sum"])))) < (np.tanh((np.where(0.318310 > -1, data["flux_ratio_sq_sum"], data["flux_err_min"] )))))*1.)) +
            1.0*np.tanh((-1.0*(((((((np.where(data["flux_err_mean"]>0, data["flux_err_mean"], (1.0) )) < (((data["flux_ratio_sq_sum"]) / 2.0)))*1.)) * 2.0))))) +
            1.0*np.tanh((((np.where(data["flux_err_max"] > -1, data["flux_ratio_sq_sum"], (8.29890346527099609) )) > ((((8.29890346527099609)) + (np.where(3.0 > -1, data["flux_by_flux_ratio_sq_sum"], data["flux_std"] )))))*1.)) +
            0.999511*np.tanh(np.where(data["flux_ratio_sq_sum"] > -1, (((np.where(data["flux_err_skew"] > -1, 3.141593, data["flux_dif3"] )) < (np.tanh((data["flux_ratio_sq_sum"]))))*1.), data["flux_dif3"] )) +
            1.0*np.tanh((-1.0*(((((np.where(3.141593<0, 3.141593, 3.0 )) < (np.maximum(((data["flux_err_mean"])), ((data["flux_mean"])))))*1.))))) +
            1.0*np.tanh((-1.0*((np.minimum((((((0.0)) + (0.0)))), (((((2.35908198356628418)) - ((((0.0) < (0.0))*1.)))))))))) +
            0.927699*np.tanh(np.minimum(((0.0)), ((np.where(((((data["flux_ratio_sq_sum"]) * 2.0)) - (data["flux_by_flux_ratio_sq_sum"])) > -1, ((1.570796) - (data["flux_by_flux_ratio_sq_sum"])), -1.0 ))))) +
            1.0*np.tanh(np.where(data["flux_err_std"]>0, (((((data["flux_err_skew"]) > (data["flux_ratio_sq_sum"]))*1.)) / 2.0), (((data["flux_mean"]) < (data["flux_err_min"]))*1.) )) +
            1.0*np.tanh(np.where(((data["flux_err_std"]) - (data["flux_min"]))<0, ((data["flux_err_std"]) - (data["flux_err_max"])), (-1.0*((((((2.70001244544982910)) < (data["flux_err_skew"]))*1.)))) )) +
            1.0*np.tanh((((np.tanh((2.0))) < (np.where(data["flux_w_mean"]<0, ((data["flux_err_median"]) * (data["flux_w_mean"])), (-1.0*((((data["flux_err_skew"]) / 2.0)))) )))*1.)) +
            0.948217*np.tanh((((np.tanh(((((data["flux_err_skew"]) > (0.636620))*1.)))) > (np.where(data["flux_err_max"] > -1, (((data["flux_max"]) < (0.0))*1.), data["flux_by_flux_ratio_sq_sum"] )))*1.)) +
            1.0*np.tanh((((((((data["flux_ratio_sq_sum"]) > (((data["flux_by_flux_ratio_sq_sum"]) / 2.0)))*1.)) / 2.0)) * ((-1.0*(((((data["flux_err_std"]) < (data["flux_err_skew"]))*1.))))))) +
            1.0*np.tanh(np.where(data["flux_err_max"] > -1, np.where(1.570796>0, (0.05156994983553886), data["flux_median"] ), np.tanh((np.where(data["flux_err_min"]<0, data["flux_mean"], (5.46952962875366211) ))) )) +
            0.783097*np.tanh(((((0.0)) < ((((data["flux_mean"]) < (np.where(data["flux_err_std"]>0, data["flux_by_flux_ratio_sq_sum"], np.minimum(((data["flux_err_skew"])), ((data["flux_err_min"]))) )))*1.)))*1.)) +
            1.0*np.tanh(np.where(data["flux_median"] > -1, 0.0, np.minimum((((((data["flux_diff"]) + (np.tanh((0.0))))/2.0))), ((((data["flux_mean"]) / 2.0)))) )) +
            0.824133*np.tanh((-1.0*((np.where((8.0)<0, data["flux_ratio_sq_sum"], ((np.tanh(((((data["flux_err_median"]) > (((data["flux_ratio_sq_sum"]) / 2.0)))*1.)))) / 2.0) ))))) +
            0.740596*np.tanh((((np.where((-1.0*((data["flux_err_mean"])))>0, (((data["flux_err_skew"]) < (2.0))*1.), data["flux_by_flux_ratio_sq_sum"] )) < ((((-1.0) + (data["flux_err_median"]))/2.0)))*1.)) +
            0.960918*np.tanh(((((np.minimum(((data["flux_ratio_sq_sum"])), ((data["flux_mean"])))) / 2.0)) * ((((data["flux_err_min"]) < (np.minimum(((0.0)), ((data["flux_mean"])))))*1.)))) +
            0.883732*np.tanh(((data["flux_mean"]) * (np.where(2.0<0, -1.0, (((3.0) < (data["flux_std"]))*1.) )))) +
            1.0*np.tanh((-1.0*(((((data["flux_by_flux_ratio_sq_sum"]) > ((((data["flux_w_mean"]) + (2.0))/2.0)))*1.))))) +
            1.0*np.tanh((((np.where(3.141593<0, 1.0, ((((0.0)) > (0.0))*1.) )) > (3.141593))*1.)) +
            1.0*np.tanh(np.tanh((np.where(1.0 > -1, ((((5.32464075088500977)) < ((((-1.0) > (1.0))*1.)))*1.), 0.0 )))) +
            1.0*np.tanh(np.minimum((((((np.minimum(((data["flux_ratio_sq_sum"])), ((0.0)))) > (data["flux_std"]))*1.))), (((((data["flux_ratio_sq_sum"]) < (data["flux_mean"]))*1.))))) +
            1.0*np.tanh(((data["hostgal_photoz_err"]) * ((((data["flux_err_max"]) + ((-1.0*((((((-1.0*((((((data["hostgal_photoz_err"]) / 2.0)) / 2.0))))) < (data["flux_err_max"]))*1.))))))/2.0)))) +
            1.0*np.tanh((((np.where(data["flux_dif2"]<0, data["flux_err_skew"], ((data["flux_dif3"]) * ((((0.12584212422370911)) / 2.0))) )) > (1.570796))*1.)) +
            0.909624*np.tanh(((data["flux_err_mean"]) - (np.where(data["flux_w_mean"]>0, data["flux_err_min"], data["flux_err_std"] )))) +
            1.0*np.tanh((-1.0*((np.where((((data["flux_ratio_sq_sum"]) < (data["flux_median"]))*1.)>0, 0.0, 0.318310 ))))) +
            1.0*np.tanh(((((((((((((data["flux_w_mean"]) / 2.0)) / 2.0)) < (data["flux_err_min"]))*1.)) < (((data["flux_ratio_sq_sum"]) / 2.0)))*1.)) / 2.0)) +
            0.993649*np.tanh(((((0.0) - (((np.where(data["flux_ratio_sq_sum"]>0, data["flux_by_flux_ratio_sq_sum"], np.where(data["flux_by_flux_ratio_sq_sum"] > -1, 0.318310, (0.63801902532577515) ) )) / 2.0)))) / 2.0)))

def GPy(data):
    return (1.0*np.tanh((((((((((((((-1.0*((data["flux_w_mean"])))) * 2.0)) * 2.0)) + (np.tanh((-1.0))))) + (data["flux_w_mean"]))) * 2.0)) * 2.0)) +
            1.0*np.tanh(((np.where(data["flux_by_flux_ratio_sq_sum"]<0, data["flux_ratio_sq_sum"], data["flux_err_std"] )) - (((((data["flux_w_mean"]) * 2.0)) * 2.0)))) +
            1.0*np.tanh(np.where(data["flux_mean"]<0, np.where(data["flux_by_flux_ratio_sq_sum"]<0, 1.570796, ((data["flux_err_max"]) * 2.0) ), ((data["flux_err_skew"]) + (data["flux_min"])) )) +
            1.0*np.tanh(np.where(data["flux_by_flux_ratio_sq_sum"]<0, np.where(data["flux_err_median"]<0, data["flux_dif3"], ((0.0) - (data["flux_dif3"])) ), data["flux_err_mean"] )) +
            1.0*np.tanh(((((np.where(data["flux_err_mean"]<0, np.where(data["flux_err_skew"]<0, data["flux_dif3"], (0.0) ), data["flux_err_skew"] )) - (data["flux_by_flux_ratio_sq_sum"]))) * 2.0)) +
            1.0*np.tanh(((np.where(data["flux_dif3"] > -1, ((np.where(data["flux_ratio_sq_sum"] > -1, (5.0), (-1.0*(((5.0)))) )) * 2.0), data["flux_err_min"] )) * 2.0)) +
            1.0*np.tanh((((((-1.0*((np.where(np.minimum(((data["flux_by_flux_ratio_sq_sum"])), ((1.0)))<0, data["flux_dif3"], data["flux_err_median"] ))))) * 2.0)) * (data["flux_err_mean"]))) +
            1.0*np.tanh((-1.0*((np.where(data["flux_ratio_sq_sum"] > -1, np.where(data["flux_ratio_sq_sum"] > -1, np.minimum(((data["flux_ratio_sq_sum"])), ((data["flux_mean"]))), data["flux_dif3"] ), data["flux_dif3"] ))))) +
            1.0*np.tanh(np.where(data["flux_err_median"]<0, 0.318310, ((((np.where(data["flux_ratio_sq_sum"]<0, data["flux_ratio_sq_sum"], 3.141593 )) * (data["flux_w_mean"]))) + (data["flux_ratio_sq_sum"])) )) +
            1.0*np.tanh(np.where(np.where(data["flux_dif3"]<0, 2.0, ((data["flux_err_mean"]) * 2.0) ) > -1, data["flux_err_skew"], (-1.0*((data["flux_err_skew"]))) )) +
            0.996580*np.tanh(np.where(((data["flux_ratio_sq_sum"]) + (data["flux_err_mean"]))<0, ((-1.0) + (data["flux_ratio_sq_sum"])), (((1.570796) > (data["flux_mean"]))*1.) )) +
            1.0*np.tanh((-1.0*((((data["flux_std"]) - (np.where(data["flux_ratio_sq_sum"] > -1, (-1.0*((data["flux_err_skew"]))), data["flux_err_skew"] ))))))) +
            1.0*np.tanh(np.where(np.minimum(((data["flux_ratio_sq_sum"])), ((np.minimum(((((data["flux_by_flux_ratio_sq_sum"]) - (data["flux_std"])))), ((data["flux_mean"])))))) > -1, data["flux_mean"], (-1.0*((data["flux_mean"]))) )) +
            1.0*np.tanh((-1.0*((np.where(data["flux_mean"] > -1, np.where(data["flux_ratio_sq_sum"] > -1, data["flux_mean"], ((3.0) * (data["flux_dif3"])) ), data["flux_mean"] ))))) +
            1.0*np.tanh(np.where(data["flux_err_mean"] > -1, np.where(1.0 > -1, np.where(data["flux_err_mean"]>0, data["flux_by_flux_ratio_sq_sum"], data["flux_dif3"] ), data["flux_dif3"] ), (-1.0*((data["flux_by_flux_ratio_sq_sum"]))) )) +
            1.0*np.tanh(np.where(np.where(data["flux_err_mean"]<0, data["flux_dif2"], data["flux_err_std"] )<0, 0.636620, (-1.0*((data["flux_dif2"]))) )) +
            1.0*np.tanh(np.where(data["flux_err_median"] > -1, (((data["flux_err_mean"]) + (np.where(0.318310 > -1, data["flux_min"], data["flux_err_median"] )))/2.0), data["flux_min"] )) +
            1.0*np.tanh(np.where(data["flux_ratio_sq_sum"] > -1, data["flux_dif3"], (((data["flux_err_max"]) < (np.tanh((((0.0) / 2.0)))))*1.) )) +
            1.0*np.tanh(np.where(data["flux_ratio_sq_sum"] > -1, np.where(data["flux_err_max"] > -1, np.minimum(((0.0)), ((0.318310))), ((data["flux_std"]) * 2.0) ), (-1.0*((data["flux_dif3"]))) )) +
            1.0*np.tanh(((np.where(data["flux_by_flux_ratio_sq_sum"] > -1, data["flux_mean"], data["flux_err_skew"] )) * (np.tanh((((((((data["flux_err_median"]) > (2.0))*1.)) + (data["flux_min"]))/2.0)))))) +
            1.0*np.tanh((((-1.0*(((((np.maximum((((((data["flux_median"]) > (1.570796))*1.))), ((data["flux_err_skew"])))) > ((((2.0) > (data["flux_median"]))*1.)))*1.))))) * 2.0)) +
            1.0*np.tanh(((((data["flux_err_max"]) * (np.minimum(((data["flux_err_skew"])), ((data["flux_err_skew"])))))) * (np.where(data["flux_std"]>0, ((data["flux_max"]) / 2.0), data["flux_dif3"] )))) +
            1.0*np.tanh(((((data["flux_err_skew"]) * (data["flux_err_median"]))) * (data["flux_dif2"]))) +
            1.0*np.tanh((((3.0) < (np.where(data["flux_ratio_sq_sum"]>0, (((data["flux_err_median"]) > (1.0))*1.), ((data["flux_err_mean"]) + (data["flux_err_mean"])) )))*1.)) +
            1.0*np.tanh((-1.0*(((((data["flux_err_min"]) > (((1.0) + (np.where(data["flux_ratio_sq_sum"] > -1, (1.0), (-1.0*((1.0))) )))))*1.))))) +
            1.0*np.tanh(((data["flux_err_min"]) * (np.where(data["flux_err_min"] > -1, ((0.0) * (0.0)), data["flux_dif3"] )))) +
            1.0*np.tanh(((np.where(data["det_flux_err_mean"]>0, data["hostgal_photoz"], ((data["det_flux_err_mean"]) + (np.where(data["det_flux_err_mean"]<0, data["flux_ratio_sq_sum"], 3.0 ))) )) * 2.0)) +
            1.0*np.tanh(np.where(data["flux_err_min"] > -1, (-1.0*(((((data["flux_by_flux_ratio_sq_sum"]) < (((data["flux_err_median"]) / 2.0)))*1.)))), ((data["flux_err_median"]) * (data["flux_by_flux_ratio_sq_sum"])) )) +
            1.0*np.tanh(np.where(data["flux_err_min"]>0, data["flux_ratio_sq_sum"], np.where(data["flux_err_min"] > -1, data["flux_dif2"], (((data["flux_ratio_sq_sum"]) > (data["flux_dif2"]))*1.) ) )) +
            1.0*np.tanh((-1.0*(((((data["flux_dif2"]) + (np.minimum(((data["flux_dif2"])), ((((((((((1.0)) / 2.0)) > (data["flux_err_max"]))*1.)) - (data["flux_err_max"])))))))/2.0))))) +
            1.0*np.tanh(np.minimum(((np.maximum(((((data["flux_err_skew"]) * 2.0))), ((data["flux_mean"]))))), ((((1.0) - (np.maximum(((data["flux_ratio_sq_sum"])), ((data["flux_err_skew"]))))))))) +
            0.946751*np.tanh(np.maximum((((((((-1.0) > (data["flux_err_mean"]))*1.)) * 2.0))), (((((-1.0) > (data["flux_mean"]))*1.))))) +
            1.0*np.tanh(np.where(data["flux_err_median"] > -1, 0.0, (-1.0*((np.where(np.where(0.0<0, 0.0, data["flux_dif3"] )<0, 0.0, data["flux_dif3"] )))) )) +
            1.0*np.tanh(np.where(((((7.71504354476928711)) < (((((12.04613590240478516)) < ((6.0)))*1.)))*1.)<0, 0.0, 0.0 )) +
            1.0*np.tanh(np.minimum(((((np.maximum(((1.570796)), ((np.maximum(((1.570796)), ((0.0))))))) + (data["flux_min"])))), ((0.0)))) +
            1.0*np.tanh(np.minimum((((((0.318310) < (data["flux_ratio_sq_sum"]))*1.))), ((np.tanh((np.where(data["flux_diff"]<0, 3.141593, data["flux_err_max"] ))))))) +
            1.0*np.tanh((-1.0*(((((2.0) < (np.where(1.0<0, (((2.0) < (data["flux_std"]))*1.), data["flux_err_max"] )))*1.))))) +
            1.0*np.tanh((-1.0*(((((0.318310) > (np.where(data["flux_err_min"]>0, ((data["flux_err_max"]) / 2.0), np.maximum(((1.570796)), ((1.570796))) )))*1.))))) +
            0.845139*np.tanh(((((((((np.minimum(((0.318310)), ((0.0)))) + (1.0))/2.0)) > ((-1.0*((data["flux_err_mean"])))))*1.)) / 2.0)) +
            1.0*np.tanh((-1.0*((((((((3.0) < (data["flux_err_std"]))*1.)) + ((((data["flux_err_std"]) > (((0.0) * (((0.0) / 2.0)))))*1.)))/2.0))))) +
            1.0*np.tanh((((np.where(data["flux_err_std"]<0, data["flux_ratio_sq_sum"], np.where(data["flux_max"]<0, 3.141593, data["flux_mean"] ) )) < (((data["flux_err_max"]) / 2.0)))*1.)) +
            1.0*np.tanh(np.minimum(((np.where(data["flux_diff"] > -1, 0.0, data["flux_err_max"] ))), ((np.where(data["flux_min"]<0, 3.141593, data["flux_mean"] ))))) +
            1.0*np.tanh(np.where(3.0 > -1, 0.0, -1.0 )) +
            0.991207*np.tanh((((0.0) < ((-1.0*((((((((3.0) > ((0.0)))*1.)) < (-1.0))*1.))))))*1.)) +
            1.0*np.tanh((((((((-1.0*((1.570796)))) > (data["flux_max"]))*1.)) > ((((data["flux_err_std"]) > (((0.0) * ((-1.0*((1.570796)))))))*1.)))*1.)) +
            1.0*np.tanh(((((data["flux_dif3"]) * (np.where(data["flux_min"] > -1, data["flux_mean"], (-1.0*((data["flux_mean"]))) )))) / 2.0)) +
            1.0*np.tanh(((np.where(((data["flux_ratio_sq_sum"]) + (data["flux_ratio_sq_sum"])) > -1, data["flux_dif3"], (((data["flux_err_max"]) < (0.0))*1.) )) / 2.0)) +
            0.954568*np.tanh((-1.0*((np.where(((np.where(data["flux_ratio_sq_sum"]>0, 0.636620, data["flux_max"] )) / 2.0)>0, data["flux_dif2"], 0.318310 ))))) +
            1.0*np.tanh((((((data["flux_ratio_sq_sum"]) > ((((np.where(data["flux_ratio_sq_sum"] > -1, 0.318310, ((0.0) / 2.0) )) + (((data["flux_ratio_sq_sum"]) / 2.0)))/2.0)))*1.)) / 2.0)) +
            1.0*np.tanh((((np.where(data["flux_median"] > -1, ((((0.0) * 2.0)) * 2.0), data["flux_err_mean"] )) < (0.0))*1.)) +
            1.0*np.tanh(((((((((data["flux_err_median"]) * 2.0)) + (data["flux_err_min"]))) * (((data["flux_err_median"]) - (data["flux_err_min"]))))) * (0.636620))) +
            0.999511*np.tanh(np.minimum(((0.0)), ((np.where(data["flux_mean"]>0, data["flux_err_median"], (((data["flux_err_median"]) < (0.0))*1.) ))))) +
            1.0*np.tanh(((((1.0)) < (((np.where(data["flux_max"]>0, (((0.06566764414310455)) * 2.0), data["flux_ratio_sq_sum"] )) * 2.0)))*1.)) +
            1.0*np.tanh(((data["flux_mean"]) * ((((((data["flux_err_max"]) / 2.0)) > (np.where(np.minimum(((3.0)), ((data["flux_err_median"])))>0, data["flux_err_median"], 1.570796 )))*1.)))) +
            1.0*np.tanh((((((((data["flux_err_min"]) < (data["flux_mean"]))*1.)) / 2.0)) * (data["flux_dif3"]))) +
            1.0*np.tanh(((((np.where(data["flux_err_median"] > -1, (((1.570796) < (data["flux_dif3"]))*1.), ((3.0) * (data["flux_dif3"])) )) * (-1.0))) / 2.0)) +
            1.0*np.tanh((((((data["flux_err_mean"]) * (((data["flux_std"]) + (data["flux_err_median"]))))) < (0.0))*1.)) +
            1.0*np.tanh(np.where(data["flux_err_skew"]<0, 0.0, (((-1.0*(((((data["flux_err_mean"]) > (data["flux_err_skew"]))*1.))))) / 2.0) )) +
            1.0*np.tanh(np.minimum(((((0.0) * ((((((data["flux_max"]) * (data["flux_max"]))) < (data["flux_ratio_sq_sum"]))*1.))))), ((((data["flux_err_max"]) * (data["flux_max"])))))) +
            1.0*np.tanh(((((np.where(data["flux_err_mean"]>0, data["flux_by_flux_ratio_sq_sum"], (((0.0)) - ((2.0))) )) / 2.0)) * (((((2.0)) < (data["flux_err_skew"]))*1.)))) +
            1.0*np.tanh(((((data["flux_dif3"]) * (np.where(data["flux_diff"]>0, (-1.0*((data["flux_dif2"]))), ((0.318310) + (data["flux_dif2"])) )))) / 2.0)) +
            0.999511*np.tanh(((np.where(((((data["flux_mean"]) * 2.0)) * 2.0) > -1, (((0.318310) > (((data["flux_mean"]) * 2.0)))*1.), data["flux_diff"] )) / 2.0)) +
            1.0*np.tanh((((((data["flux_err_skew"]) > ((((((((0.0) < (data["flux_ratio_sq_sum"]))*1.)) + (1.570796))) + (0.0))))*1.)) * (data["flux_mean"]))) +
            1.0*np.tanh((((((data["flux_err_min"]) < (-1.0))*1.)) * (0.0))) +
            1.0*np.tanh((((((3.141593) * ((((((data["flux_min"]) > ((3.84652471542358398)))*1.)) * 2.0)))) > ((3.84652471542358398)))*1.)) +
            1.0*np.tanh(np.minimum(((0.636620)), ((((data["flux_err_median"]) - (np.where((10.0)<0, data["flux_err_min"], data["flux_err_min"] ))))))) +
            1.0*np.tanh((((np.where(data["flux_by_flux_ratio_sq_sum"]<0, data["flux_err_mean"], data["flux_err_median"] )) > (np.where(data["flux_ratio_sq_sum"] > -1, data["flux_err_median"], 1.0 )))*1.)) +
            0.877382*np.tanh(((np.minimum(((np.maximum(((data["flux_err_skew"])), ((data["flux_by_flux_ratio_sq_sum"]))))), ((np.minimum(((np.minimum((((0.0))), ((0.0))))), ((((data["flux_ratio_sq_sum"]) / 2.0)))))))) / 2.0)) +
            1.0*np.tanh((((((((((-1.0) / 2.0)) > (data["flux_err_skew"]))*1.)) * (data["flux_dif2"]))) / 2.0)) +
            1.0*np.tanh((((np.where(data["flux_err_min"] > -1, 3.141593, data["flux_dif3"] )) < (np.minimum(((-1.0)), ((np.tanh((data["flux_err_skew"])))))))*1.)) +
            1.0*np.tanh((((np.where(data["flux_by_flux_ratio_sq_sum"]>0, data["flux_mean"], 1.570796 )) < (((-1.0) / 2.0)))*1.)) +
            1.0*np.tanh(((data["flux_err_min"]) * ((-1.0*((np.where(data["flux_ratio_sq_sum"]<0, np.maximum(((0.0)), ((data["flux_err_skew"]))), 0.0 ))))))) +
            1.0*np.tanh(((np.tanh((((((((np.tanh(((((data["flux_err_max"]) > (0.636620))*1.)))) + (data["flux_diff"]))/2.0)) > (0.636620))*1.)))) / 2.0)) +
            1.0*np.tanh((((np.where(data["flux_by_flux_ratio_sq_sum"]<0, data["flux_err_median"], np.tanh(((((-1.0) + ((9.0)))/2.0))) )) > (1.570796))*1.)) +
            0.999511*np.tanh((-1.0*(((((2.0) < (np.maximum(((data["flux_err_std"])), ((data["flux_std"])))))*1.))))) +
            1.0*np.tanh((-1.0*(((((-1.0*(((-1.0*((np.tanh((((np.where(data["flux_err_std"]<0, data["flux_dif3"], 0.0 )) / 2.0)))))))))) / 2.0))))) +
            1.0*np.tanh(((data["flux_err_skew"]) * (np.minimum((((((((data["flux_ratio_sq_sum"]) < (data["flux_err_skew"]))*1.)) * (data["flux_diff"])))), ((np.maximum(((data["flux_min"])), ((data["flux_ratio_sq_sum"]))))))))) +
            0.927699*np.tanh((-1.0*((np.where(data["flux_err_max"] > -1, ((((10.0)) < (((((7.30266284942626953)) < (((((10.0)) > ((13.33415126800537109)))*1.)))*1.)))*1.), data["flux_dif3"] ))))) +
            1.0*np.tanh(((0.0) * (3.0))) +
            1.0*np.tanh(((np.where((14.93928241729736328)<0, ((((6.0)) < (1.0))*1.), (((((data["flux_err_skew"]) + (0.0))/2.0)) / 2.0) )) / 2.0)) +
            1.0*np.tanh((((3.0) < (((np.where(data["flux_err_skew"]<0, data["flux_mean"], ((((((6.0)) < ((1.0)))*1.)) + ((1.0))) )) * 2.0)))*1.)) +
            0.948217*np.tanh((((((-1.0) + ((((data["flux_err_median"]) < (np.where(2.0<0, ((((9.47109317779541016)) > (data["flux_max"]))*1.), 0.318310 )))*1.)))/2.0)) / 2.0)) +
            1.0*np.tanh(((3.141593) * ((((((((((((3.0)) > ((7.0)))*1.)) * (0.0))) > ((3.0)))*1.)) * 2.0)))) +
            1.0*np.tanh(np.where(data["flux_err_mean"] > -1, ((np.where(data["flux_std"] > -1, 0.0, data["flux_dif3"] )) / 2.0), data["flux_dif3"] )) +
            0.783097*np.tanh((((data["flux_by_flux_ratio_sq_sum"]) > ((((data["flux_mean"]) + (np.maximum(((data["flux_err_min"])), ((3.141593)))))/2.0)))*1.)) +
            1.0*np.tanh((-1.0*(((((data["flux_mean"]) > (((1.570796) + (np.where(data["flux_w_mean"] > -1, data["flux_ratio_sq_sum"], data["flux_w_mean"] )))))*1.))))) +
            0.824133*np.tanh(np.tanh(((((((np.minimum(((0.318310)), ((((0.318310) / 2.0))))) + ((((((data["flux_err_mean"]) / 2.0)) > (0.318310))*1.)))/2.0)) / 2.0)))) +
            0.740596*np.tanh(((((((((0.0)) * 2.0)) > (np.where(data["flux_err_skew"] > -1, (7.78979110717773438), ((data["flux_err_max"]) / 2.0) )))*1.)) * (2.0))) +
            0.960918*np.tanh((-1.0*((np.where(np.maximum(((data["flux_dif3"])), ((0.0)))>0, ((0.318310) / 2.0), ((data["flux_err_skew"]) * (0.318310)) ))))) +
            0.883732*np.tanh(((((data["flux_dif3"]) / 2.0)) * (np.maximum(((np.maximum(((0.0)), (((((data["flux_err_min"]) < (data["flux_err_mean"]))*1.)))))), ((np.tanh((data["flux_err_std"])))))))) +
            1.0*np.tanh(np.minimum(((((data["flux_err_max"]) - (data["flux_err_std"])))), (((((data["flux_median"]) > (((data["flux_by_flux_ratio_sq_sum"]) - (data["flux_err_std"]))))*1.))))) +
            1.0*np.tanh(((np.where(data["flux_ratio_sq_sum"]>0, 0.0, np.where(((data["flux_err_median"]) - (data["flux_ratio_sq_sum"]))>0, (-1.0*((data["flux_dif3"]))), data["flux_dif3"] ) )) / 2.0)) +
            1.0*np.tanh((((((data["flux_err_median"]) * ((((((data["flux_err_median"]) / 2.0)) + (data["flux_dif3"]))/2.0)))) > ((((3.0) + (data["flux_w_mean"]))/2.0)))*1.)) +
            1.0*np.tanh((((3.0) < (0.0))*1.)) +
            1.0*np.tanh(np.minimum(((0.0)), (((((0.636620) + (0.0))/2.0))))) +
            1.0*np.tanh(np.where(data["flux_std"] > -1, (((data["flux_dif2"]) < ((-1.0*(((4.49028682708740234))))))*1.), ((data["flux_dif2"]) * (-1.0)) )) +
            0.909624*np.tanh(((np.where(np.where(data["flux_by_flux_ratio_sq_sum"]<0, data["flux_ratio_sq_sum"], (((data["flux_by_flux_ratio_sq_sum"]) < (3.141593))*1.) ) > -1, 0.0, data["flux_by_flux_ratio_sq_sum"] )) / 2.0)) +
            1.0*np.tanh((((((np.where(data["flux_err_min"] > -1, 2.0, data["flux_dif2"] )) < (-1.0))*1.)) * 2.0)) +
            1.0*np.tanh(((3.0) * (np.tanh(((-1.0*((0.0)))))))) +
            0.993649*np.tanh((((3.141593) < (np.where(((((8.0)) < ((((((data["flux_err_max"]) > ((7.0)))*1.)) / 2.0)))*1.)>0, 0.0, data["flux_err_min"] )))*1.)))

In [ ]:
myfilter = (traindata.ddf==0)&(traindata.hostgal_specz==0)
cm = plt.cm.get_cmap('RdYlBu')
fig, axes = plt.subplots(1, 1, figsize=(10, 10))
sc = axes.scatter(GPx(alldata[myfilter]),
                  GPy(alldata[myfilter]),
                  alpha=1,
                  c=(traindata[myfilter].target.values),
                  cmap=cm,
                  s=1)
cbar = fig.colorbar(sc, ax=axes)
cbar.set_label('Target')
_ = axes.set_title("Clustering colored by target")



In [ ]:
myfilter = (traindata.ddf==0)&(traindata.hostgal_specz!=0)
cm = plt.cm.get_cmap('RdYlBu')
fig, axes = plt.subplots(1, 1, figsize=(10, 10))
sc = axes.scatter(GPx(alldata[myfilter]),
                  GPy(alldata[myfilter]),
                  alpha=1,
                  c=(traindata[myfilter].target.values),
                  cmap=cm,
                  s=1)
cbar = fig.colorbar(sc, ax=axes)
cbar.set_label('Target')
_ = axes.set_title("Clustering colored by target")

In [ ]:
myfilter = (traindata.ddf==1)&(traindata.hostgal_specz==0)
cm = plt.cm.get_cmap('RdYlBu')
fig, axes = plt.subplots(1, 1, figsize=(10, 10))
sc = axes.scatter(GPx(alldata[myfilter]),
                  GPy(alldata[myfilter]),
                  alpha=1,
                  c=(traindata[myfilter].target.values),
                  cmap=cm,
                  s=1)
cbar = fig.colorbar(sc, ax=axes)
cbar.set_label('Target')
_ = axes.set_title("Clustering colored by target")

In [ ]:
myfilter = (traindata.ddf==1)&(traindata.hostgal_specz!=0)
cm = plt.cm.get_cmap('RdYlBu')
fig, axes = plt.subplots(1, 1, figsize=(10, 10))
sc = axes.scatter(GPx(alldata[myfilter]),
                  GPy(alldata[myfilter]),
                  alpha=1,
                  c=(traindata[myfilter].target.values),
                  cmap=cm,
                  s=1)
cbar = fig.colorbar(sc, ax=axes)
cbar.set_label('Target')
_ = axes.set_title("Clustering colored by target")